Datasette: https://docs.datasette.io/
        
sqlite-utils: https://sqlite-utils.readthedocs.io/

Before running this notebook I did:

    pip install httpx sqlite-utils

In [1]:
url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_week.geojson"

In [3]:
print(url)

https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_week.geojson


In [4]:
import httpx

In [5]:
data = httpx.get(url).json()

In [7]:
def feature(f):
    d = f["properties"]
    d["latitude"] = f["geometry"]["coordinates"][1]
    d["longitude"] = f["geometry"]["coordinates"][0]
    d["id"] = f["id"]
    return d

In [8]:
feature(data["features"][0])

{'mag': 1.95,
 'place': '12km WSW of Niland, CA',
 'time': 1602083376960,
 'updated': 1602083520739,
 'tz': None,
 'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ci39655376',
 'detail': 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/detail/ci39655376.geojson',
 'felt': None,
 'cdi': None,
 'mmi': None,
 'alert': None,
 'status': 'automatic',
 'tsunami': 0,
 'sig': 58,
 'net': 'ci',
 'code': '39655376',
 'ids': ',ci39655376,',
 'sources': ',ci,',
 'types': ',nearby-cities,origin,phase-data,scitech-link,',
 'nst': 11,
 'dmin': 0.008703,
 'rms': 0.25,
 'gap': 92,
 'magType': 'ml',
 'type': 'earthquake',
 'title': 'M 2.0 - 12km WSW of Niland, CA',
 'latitude': 33.1801682,
 'longitude': -115.6323318,
 'id': 'ci39655376'}

In [9]:
import sqlite_utils

In [10]:
db = sqlite_utils.Database("/tmp/earthquakes.db")

In [12]:
db["quakes"].insert_all((feature(f) for f in data["features"]), pk="id")

<Table quakes (mag, place, time, updated, tz, url, detail, felt, cdi, mmi, alert, status, tsunami, sig, net, code, ids, sources, types, nst, dmin, rms, gap, magType, type, title, latitude, longitude, id)>

In [13]:
db["quakes"].enable_fts(["place"])

<Table quakes (mag, place, time, updated, tz, url, detail, felt, cdi, mmi, alert, status, tsunami, sig, net, code, ids, sources, types, nst, dmin, rms, gap, magType, type, title, latitude, longitude, id)>

Then run this to open Datasette:
    
    datasette /tmp/earthquakes.db

Installation instructions for Datasette here: https://docs.datasette.io/en/stable/installation.html

Short version: `pip install datasette` or `brew install datasette`

I installed the mapping plugin using `datasette install datasette-cluster-map`